Кобзарь О.С. Хабибуллин Р.А. 2019 г.

## Данное решение находится в стадии разработки. Нет  учета изменения свойств флюида в затрубном пространстве. Нет уверенности в правильности формул.

Источник:

Fluid Flow and Heat Transfer in Wellbores A.R. Hasan and C.S. Kabir 2002 175 pp.; Softcover ISBN: 978-1-55563-094-2
Society of Petroleum Engineers

### КРТ. Температурная корреляция Хасан Кабир
#### Последовательный расчет градиента давления (Беггз Брил) и градиента температуры

## Основное уравнение

В отличие от выражение температуры флюида в проинтегрированном виде:

$$T_f = (T_{ei} - g_G L sin\theta) + (T_{fi} - T_{ei})e^{-L/A} + A (1 - e^{-L /A })(g_G sin\theta - \frac{gsin\theta}{C_p} + F_c)$$

Уравнение градиента температуры имеет следующий вид:

$$\frac{dT_f}{dz} = C_J\frac{dp}{dz} - \frac{Q}{wc_p} - \frac{g sin \alpha}{c_p} -  \frac{vdv}{c_p dz}$$

части выражения учитывают изменение температуры:

$C_J\frac{dp}{dz}$ - за счет эффекта Джоуля Томпсона

$\frac{Q}{wc_p}$ - за счет теплопереноса

$\frac{g sin \alpha}{c_p}$ - изменения потенциальной энергии

$\frac{vdv}{c_p dz}$ - изменения кинетической энергии

## Подготовка к вычислению

Чтобы вычислить градиент температуры в определенных термобарических условиях сперва необходимо вычислить PVT свойства (в том числе теплофизические) флюидов, рассчитать параметры многофазного потока и общие свойства смеси, градиент давления.

Также стоит определится, производится ли расчет участка скважины с НКТ (учет затрубного пространства  -  `Pipe().section_casing = False`) или участка от забоя до приема (ГЖС движется внутри обсадной трубы - `Pipe().section_casing = True`). В первом случае также стоит определиться с флюидом, который находится в затрубном пространстве - рассчитать его свойства. Подробнее об этом смотрите в разделе uPVT

## Вычисления 

### Расчет вынужденной конвекции в трубе (НКТ или ОТ - по которой движется смесь)

$$N_{Re_n} = \frac{\rho_n v_m d _{ti}}{\mu_n}$$

$$N_{Pr_n} = \frac{\mu_n C _{P_n}}{k_n}$$

$$N_{Nu} = 0.023 N_{Re_n} ^ {0.8} N_{Pr_n} ^ {0.3}$$

$$h_f = \frac{N_{Nu}k_n}{d_{ti}}$$

### Естественная конвекция в затрубном пространстве (только для расчета НКТ)

Расчет коэффициента конвекции в затрубном пространстве вызывает небольшую трудность, т.к. требуется знать число Грасгофа, которое в свою очередь зависит от перепада температур в затрубном пространстве. Чтобы решить данную проблему можно воспользоваться итерационным подходом: в первый раз принять начальное приближение, потом вычислить коэффициент, температуру, тепловой поток, и снова коэффициент для конвекции - повторить до тех пор, пока не будет изменяться. В данном вопросе значительно упрощают жизнь библиотека `scipy`.

1. Расчет безразмерных чисел Прандтля и Грасгофа для затрубного пространства

$$N_{Pr} = \frac{\mu_{an} C _{P_{an}}}{k_{an}}$$

2. Положим начальное приближение для перепада температур в затрубном пространстве

$$(T_{to} - T_{ci}) = 3^{\circ}$$

и можем рассчитать:

$$N_{Gr} = \frac {(r_{ci} - r_{to})^3 \bar{\rho}_{an}^2 \beta g (T_{to} - T_{ci}) }{\bar{\mu}_{an}^2}$$

3. Рассчитаем коэффициент естественной конвекции:

$$h_{an} = \frac{0.049(N_{Gr} N_{Pr}) ^ {(1/3)} N_{Gr} ^ {0.074} k_{an}}{r_{to}ln(\frac{r_{ci}}{r_{to}})}$$

4. Скорректируем коэффициент для учета длины трубы

$$h_{an}^{*} = 0.25 h_{an}$$

5. Тепловой поток

$$ Q =\frac{ 2 \pi r_{to} U_{to} }{T_f - T_{ei}}  $$

6. Перепад температур в затрубном пространстве:

$$ \delta T_{an} = \frac{Q}{2 \pi r_{ti} h_{an}} $$



### Определение общего коэффициента теплопередачи:

1. Рассчитываем безразмерное время:

$$ t_{Dw} = \frac{k_e t}{\rho_e C_{p_e}r_{wb}^2} $$

2. и безразмерную температуру (потери тепла в окружающую среду): 

$$ T_D = ln[\exp ^{-0.2t_D} + (1.5 - 0.3719 \exp ^{-t_D})  \sqrt{t_D} ]$$

2. Определяем тепловое сопротивление элементов скважины

$$(R_{conv})_f = \frac{1}{r_{ti}h_f}$$

при расчете НКТ учитываем теплопроводность НКТ и конвективный теплообмен в затрубном пространстве:

$$(R_{cond})_t  = \frac{ln(r_{to}/r_{ti})}{k_t}$$

$$(R_{conv})_{an} = \frac{1}{r_{ci}h_{an}}$$

$$(R_{cond})_c  = \frac{ln(r_{co}/r_{ci})}{k_c}$$

$$(R_{cond})_{cem}  = \frac{ln(r_{w}/r_{co})}{k_{cem}}$$

$$(R_{cond})_{e}  = \frac{T_D}{k_{e}}$$

3. Определяем общий коэффициент теплопередачи

$$U  = \frac{1}{r_{to}}[\frac{1}{ (R_{conv})_f + (R_{cond})_t + (R_{conv})_{an} +(R_{cond})_c + (R_{cond})_{cem}+(R_{cond})_{e} }]$$


### Последние формулы:

1. Параметр релаксации:

$$L_R = \frac{2 \pi}{c_p w} [\frac{r_{to} U_{to} k_e} {k_e + (r_{to} U_{to} T_D)}] = \frac {1}{A} $$

2. Тепловой поток:

$$ Q = L_R w c_p (T_f - T_{ei}) $$

3. Подствим полученные значения в итоговую формулу

$$\frac{dT_f}{dz} = C_J\frac{dp}{dz} - \frac{Q}{wc_p} - \frac{g sin \alpha}{c_p} -  \frac{vdv}{c_p dz}$$


Импорт необходимых модулей

In [1]:
import sys
sys.path.append('../')


import uniflocpy.uWell.uPipe as Pipe
import uniflocpy.uTools.data_workflow as tool
import uniflocpy.uTools.uconst as uc

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools

init_notebook_mode(connected=True)

создание экземляра класса Pipe и хранилищ данных

In [2]:
tube = Pipe.Pipe()
tube_data = tool.Data()
fluid_flow_data = tool.Data()
temp_cor_data = tool.Data()
fluid_data = tool.Data()

Задание необходимых параметров скважины (конструкция трубы, начальные условия, свойства потока, время работы)

In [3]:
p_init_bar = 90
t_fluid_init_c = 91
t_earth_init_c = 90
q_liq_m3day = 50
watercut_perc = 20
d_tube_m = 0.130
work_time_sec = 60
geo_grad_cm = 0.027
depth_m = 3000
step_m = 10

Передача параметров скважины в класс Pipe

In [4]:
tube.fluid_flow.qliq_on_surface_m3day = q_liq_m3day
tube.fluid_flow.fw_on_surface_perc = watercut_perc
tube.fluid_flow.d_m = d_tube_m

tube.section_casing = True

tube.t_earth_init_c = t_earth_init_c
tube.time_sec = work_time_sec

Проверка для расчета градиента давления и температуры при начальных условиях

In [5]:
tube.calc_p_grad_pam(p_init_bar, t_fluid_init_c)

10055.042865614794

In [6]:
tube.calc_t_grad_cm(p_init_bar, t_fluid_init_c)

-0.00506354213955985

Создание хранилищ для основных параметров работы

In [7]:
p_list_bar = [p_init_bar]
t_list_c = [t_fluid_init_c]
h_list_m = [depth_m]
t_earth_list_c = [t_earth_init_c]
distance_from_bottomhole = [0]

Основной расчет через цикл

In [8]:
for i in range(int(depth_m / step_m)):
    if p_list_bar[-1] > 2:
        p_grad_pam = tube.calc_p_grad_pam(p_list_bar[-1], t_list_c[-1])
        t_grad_cm = tube.calc_t_grad_cm(p_list_bar[-1], t_list_c[-1])
        

        
        
        p_bar = p_list_bar[-1] - uc.Pa2bar(p_grad_pam) * step_m
        t_c = t_list_c[-1] - t_grad_cm * step_m
        t_earth_c = t_earth_list_c[-1] - geo_grad_cm * step_m
        h_m = h_list_m[-1] - step_m
        distance_m = distance_from_bottomhole[-1] + step_m
        
        
        tube_data.get_data(tube)
        fluid_flow_data.get_data(tube.fluid_flow)
        temp_cor_data.get_data(tube.temp_cor)
        fluid_data.get_data(tube.fluid_flow.fl)
        
        fluid_data.h_list.append(distance_m)
        
        h_list_m.append(h_m)
        p_list_bar.append(p_bar)
        t_list_c.append(t_c)
        t_earth_list_c.append(t_earth_c)
        distance_from_bottomhole.append(distance_m)
        
        tube.t_earth_init_c = t_earth_c
        

Функции, для удобства построения графиков

In [9]:
def plot():
    layout = dict(title =  'L(P бар, T C)',  yaxis=dict(title='Расстояние от забоя, м'))
    fig = dict(data=data, layout=layout)

    iplot(fig, filename='basic-scatter')

def trace(data, number_param):
    tracep = go.Scattergl(
        x = data.get_values(number_param),
        y = fluid_data.h_list,
        name = data.get_name(number_param),
        mode = 'markers'
    )
    return tracep

In [10]:
trace1 = go.Scattergl(
    x = t_earth_list_c,
    y = distance_from_bottomhole, 
    name = 'Геотерма, C',
    mode = 'markers'
)

trace2 = go.Scattergl(
    x = p_list_bar,
    y = distance_from_bottomhole, 
    name = 'КРД Беггз Брилл, бар',
    mode = 'markers'
)

trace3 = go.Scattergl(
    x = t_list_c,
    y = distance_from_bottomhole, 
    name = 'КРТ Хасан Кабир, С',
    mode = 'markers'
)


data = [ trace1, trace2, trace3]

plot()

In [11]:
tube_data
fluid_flow_data
temp_cor_data
fluid_data
fluid_data

In [12]:
temp_cor_data.print_all_names()

Номер 0 для получения параметра rho_earth_kgm3
Номер 1 для получения параметра cp_earth_jkgc
Номер 2 для получения параметра thermal_conduct_earth_wmc
Номер 3 для получения параметра thermal_conduct_tube_wmk
Номер 4 для получения параметра thermal_conduct_cas_wmk
Номер 5 для получения параметра thermal_conduct_cem_wmc
Номер 6 для получения параметра delta_t_an_init_c
Номер 7 для получения параметра section_casing
Номер 8 для получения параметра time_sec
Номер 9 для получения параметра p_bar
Номер 10 для получения параметра t_c
Номер 11 для получения параметра t_earth_init_c
Номер 12 для получения параметра angle_rad
Номер 13 для получения параметра d_m
Номер 14 для получения параметра r_tube_in_m
Номер 15 для получения параметра r_tube_out_m
Номер 16 для получения параметра r_cas_in_m
Номер 17 для получения параметра r_cas_out_m
Номер 18 для получения параметра r_well_m
Номер 19 для получения параметра liquid_content
Номер 20 для получения параметра mass_flowraten_kgsec
Номер 21 для по

In [13]:
trace1 = trace(temp_cor_data, 54)
trace2 = trace(temp_cor_data, 55)
trace3 = trace(temp_cor_data, 56)
trace4 = trace(temp_cor_data, 57)
trace5 = trace(temp_cor_data, 58)





data = [trace1, trace2,trace3,  trace4, trace5]

plot()

In [14]:
trace1 = trace(temp_cor_data, 45)
trace2 = trace(temp_cor_data, 46)
trace3 = trace(temp_cor_data, 47)
trace4 = trace(temp_cor_data, 48)
trace5 = trace(temp_cor_data, 49)
trace6 = trace(temp_cor_data, 50)





data = [trace1, trace2,trace3,  trace4, trace5, trace6]

plot()

TypeError: float() argument must be a string or a number, not 'NoneType'

In [ ]:
temp_cor_data.get_values(55)